# "self-querying" retrievalの使い方

「self-querying」に対応したベクトルストアについては、統合ページをご覧ください。

自己クエリ型のリトリーバー とは、その名の通り、自身にクエリを実行する能力を持つリトリーバーのことです。  
具体的には、任意の自然言語クエリを受け取り、リトリーバーがクエリを構築するためのLLMチェーンを使用して構造化クエリを生成します。  
その後、この構造化クエリを内部のベクトルストアに適用します。  

これにより、リトリーバーは次のようなことが可能になります：

- ユーザーが入力したクエリを使用して、保存されたドキュメントの内容と意味的な類似性を比較する。
- ユーザークエリから保存されたドキュメントのメタデータに基づくフィルターを抽出し、そのフィルターを実行する。

![](../../static/img/self_querying.jpg)

はじめに

デモンストレーションの目的で、`Chroma` ベクトルストアを使用します。ここでは、映画の要約を含む小さなデモ用ドキュメントセットを作成しました。

注意:

自己クエリ型リトリーバーを使用するには、`lark` パッケージをインストールしておく必要があります。

In [1]:
%pip install --upgrade --quiet  lark langchain-chroma

In [1]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

### self-querying retrieverの作成

次に、自己クエリ型リトリーバーをインスタンス化します。  
そのためには、ドキュメントがサポートするメタデータフィールドに関する情報と、ドキュメント内容の簡単な説明を事前に提供する必要があります。

In [2]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

### テストしてみる

いよいよ、リトリーバーを実際に使ってみましょう！

In [3]:
# この例ではフィルターのみを指定
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006})]

In [4]:
# この例ではクエリとフィルターを指定
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019})]

In [5]:
# この例では複合的なフィルターを指定
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979})]

In [6]:
# この例ではクエリと複合的なフィルターを指定
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

### フィルターで k を指定

自己クエリ型リトリーバーを使用して、取得するドキュメントの数（`k`）を指定することも可能です。

これを行うには、リトリーバーのコンストラクタに `enable_limit=True` を渡します。

In [7]:
retriever = SelfQueryRetriever.from_llm(
    llm,  # 使用するLLM
    vectorstore,  # ベクトルストア
    document_content_description,  # ドキュメント内容の説明
    metadata_field_info,  # メタデータフィールド情報
    enable_limit=True,  # 制限を有効化
)

# 関連するクエリのみを指定した例
retriever.invoke("What are two movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

## LCELを使ってゼロから構築する

内部の仕組みを理解し、より柔軟にカスタマイズしたい場合は、リトリーバーをゼロから構築することができます。

最初に、クエリ構築チェーンを作成します。このチェーンは、ユーザークエリを受け取り、StructuredQuery オブジェクトを生成します。  
このオブジェクトは、ユーザーが指定したフィルターを含む構造化クエリを表します。  
プロンプトと出力パーサーを作成するためのヘルパー関数を提供しています。ここでは簡潔にするために、調整可能なパラメータは無視します。

In [8]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,  # 構造化クエリの出力を解析するパーサー
    get_query_constructor_prompt,  # クエリ構築用のプロンプトを生成する関数
)

# プロンプトを生成
prompt = get_query_constructor_prompt(
    document_content_description,  # ドキュメント内容の説明
    metadata_field_info,  # メタデータフィールド情報
)

# 出力パーサーを生成
output_parser = StructuredQueryOutputParser.from_components()

# クエリ構築チェーンを作成
query_constructor = prompt | llm | output_parser


以下のコードで生成されたプロンプトを確認できます：

In [9]:
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

以下のコードを使って、クエリ構築チェーンが生成する結果を確認します：

In [10]:
query_constructor.invoke(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

StructuredQuery(query='taxi driver', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='year', value=2000)]), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)

クエリコンストラクタは、自己クエリ型リトリーバーの重要な要素です。  
優れた検索システムを作るには、クエリコンストラクタが正しく動作することを確認する必要があります。  
これには、プロンプト、プロンプト内の例、属性の説明などを調整する必要がある場合があります。  
ホテルの在庫データを使ったクエリコンストラクタの調整例については、このクックブックをご覧ください。 [check out this cookbook](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb).

構造化クエリの変換は、使用しているベクトルストアの構文に従って、汎用的なStructuredQueryオブジェクトをメタデータフィルターに変換する役割を担います。  
LangChainには、いくつかのビルトイン変換ツールが含まれています。詳細については統合セクションをご覧ください。  

以下は、Chroma用の変換ツールを使用する例です：

In [11]:
from langchain_community.query_constructors.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # クエリコンストラクタ
    vectorstore=vectorstore,              # ベクトルストア
    structured_query_translator=ChromaTranslator(),  # 構造化クエリの変換ツール
)

In [12]:
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]